In [1]:
import mne
import numpy as np
from mne.preprocessing import (ICA)
from autoreject import AutoReject
import matplotlib
matplotlib.use("TkAgg")


ModuleNotFoundError: No module named 'autoreject'

In [7]:
eeg_path = "C://Users//neuro//Documents//Git_EEG_Workshop//EEG_Workshop//Data//"  # You will need to change this location
file_name = "VEP_2024-07-05_15-17-52"
file_eeg = eeg_path + file_name + ".eeg"
file_vhdr = eeg_path + file_name + ".vhdr"
file_vmrk = eeg_path + file_name + ".vmrk"

In [8]:
raw = mne.io.read_raw_brainvision(file_vhdr)
#drop_channels = ['BIP1','BIP2','EOG','TEMP1','ACC1','ACC2','ACC3']
#raw = raw.drop_channels(drop_channels)
#raw.plot()
events_from_annot, event_dict = mne.events_from_annotations(raw)
del event_dict['Stimulus/s5']

Extracting parameters from C://Users//neuro//Documents//VEP//VEP Data//AJK-03-01-24.vhdr...
Setting channel info structure...


C:\Users\neuro\AppData\Local\Temp\ipykernel_4052\2967754227.py:1: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_brainvision(file_vhdr)


Using matplotlib as 2D backend.
Used Annotations descriptions: ['Marker/Impedance', 'New Segment/', 'Stimulus/s1', 'Stimulus/s2', 'Stimulus/s3', 'Stimulus/s5', 'Stimulus/s9007', 'Stimulus/s9008']


Channels marked as bad:
none


In [9]:
highpass = 1
lowpass = 20
notch = 60

raw_filtered = raw.load_data().filter(highpass, lowpass).notch_filter(np.arange(notch, (notch * 3), notch))
#raw_filtered = raw.resample(resample).filter(highpass, lowpass).notch_filter(np.arange(notch, (notch * 3), notch))

eeg_1020 = raw_filtered.copy().set_eeg_reference(ref_channels = 'average') # ref_channels='['Fz']'
ten_twenty_montage = mne.channels.make_standard_montage('standard_1020')
eeg_1020 = eeg_1020.set_montage(ten_twenty_montage, on_missing = 'ignore')
#del raw, raw_filtered, ten_twenty_montage
eeg_1020.info['bads'] = []
picks = mne.pick_types(eeg_1020.info, meg=False, eeg=True, stim=False, eog=False, include=[], exclude=[])

epochs = mne.Epochs(eeg_1020,
                    events=events_from_annot,
                    event_id=event_dict,
                    tmin=-0.050,
                    tmax=0.500,   #duration of stimulus or response
                    baseline=None,
                    reject=None,
                    verbose=False,
                    preload=True,
                    detrend=None,
                    event_repeated='drop')


Reading 0 ... 84538  =      0.000 ...   169.076 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 20 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 20.00 Hz
- Upper transition bandwidth: 5.00 Hz (-6 dB cutoff frequency: 22.50 Hz)
- Filter length: 1651 samples (3.302 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filte

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


In [10]:
n_interpolates = np.array([1, 4, 32])
consensus_percs = np.linspace(0, 1.0, 11)
ar = AutoReject(n_interpolates,
                consensus_percs,
                picks=picks,
                thresh_method='random_search',
                random_state=42)    #random n state
epochs_ar = ar.fit_transform(epochs);


c:\Users\neuro\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running autoreject on ch_type=eeg


100%|██████████| Creating augmented epochs : 33/33 [00:00<00:00,   90.17it/s]
100%|██████████| Computing thresholds ... : 33/33 [00:04<00:00,    6.84it/s]












100%|██████████| Repairing epochs : 162/162 [00:00<00:00,  633.21it/s]
















































100%|██████████| Repairing epochs : 162/162 [00:01<00:00,  154.31it/s]






















100%|██████████| Fold : 10/10 [00:01<00:00,    5.43it/s]




















































100%|██████████| Repairing epochs : 162/162 [00:01<00:00,  149.43it/s]






















100%|██████████| Fold : 10/10 [00:01<00:00,    6.35it/s]





















































100%|██████████| Repairing epochs : 162/162 [00:01<00:00,  142.08it/s]






















100%|██████████| Fold : 10/10 [00:00<00:00,   41.69it/s]
100%|██████████| n_interp : 3/3 [00:07<00:00,    2.42s/it]






Estimated consensus=0.50 and n_interpolate=4






















































100%|██████████| Repairing epochs : 162/162 [00:01<00:00,  145.64it/s]

Dropped 37 epochs: 0, 1, 2, 5, 7, 8, 9, 11, 12, 16, 22, 23, 26, 27, 28, 36, 39, 40, 43, 58, 65, 66, 68, 69, 72, 74, 86, 97, 99, 102, 103, 112, 129, 131, 133, 145, 161


In [11]:
ica = ICA(n_components = 16, max_iter = 'auto', random_state = 123)
ica.fit(epochs_ar)

ica_z_thresh = 1.96
epochs_clean = epochs_ar.copy()
eog_indices, eog_scores = ica.find_bads_eog(epochs_clean,
                                            ch_name=['Fp1', 'F8'],
                                            threshold=ica_z_thresh)
ica.exclude = eog_indices
ica.plot_scores(eog_scores)
ica.plot_sources(epochs_ar)
ica.plot_components()
print(eog_indices)
ica.apply(epochs_clean)
epochs_final = epochs_clean.copy()
#del eeg_1020, epochs, epochs_ar, eog_indices, eog_scores


Fitting ICA to data using 33 channels (please be patient, this may take a while)
Selecting by number: 16 components
Fitting ICA took 1.0s.
Using EOG channels: Fp1, F8
Not setting metadata
125 matching events found
No baseline correction applied
0 projection items activated
[0, 11]
Applying ICA to Epochs instance
    Transforming to ICA space (16 components)
    Zeroing out 2 ICA components
    Projecting back using 33 PCA components


In [28]:
baseline_tmin, baseline_tmax = -0.05, 0
baseline = (baseline_tmin, baseline_tmax)

VEP = epochs_final['Stimulus/s1'].apply_baseline(baseline).average()
blank = epochs_final['Stimulus/s2'].apply_baseline(baseline).average()

fig = mne.viz.plot_compare_evokeds(VEP, picks=['Oz','O1','O2'], combine="mean", show=False, time_unit="ms")
fig[0].savefig("VEP Data/"+file_name+"-VEP_Occipital")

fig = mne.viz.plot_compare_evokeds(VEP, picks=['O1','O2'], combine="mean", show=False, time_unit="ms")
fig[0].savefig("VEP Data/"+file_name+"-VEP_O1_O2")

fig = mne.viz.plot_compare_evokeds(VEP, picks=['Oz'], show=False, time_unit="ms")
fig[0].savefig("VEP Data/"+file_name+"-VEP_Oz")

SyntaxError: unterminated string literal (detected at line 14) (436900637.py, line 14)

In [27]:
evokeds = dict(
    Checkerboard=list(epochs_final['Stimulus/s1'].iter_evoked()),
    Blank=list(epochs_final['Stimulus/s2'].iter_evoked()),
)

fig = mne.viz.plot_compare_evokeds(evokeds, 
                                   colors=dict(Checkerboard="red", Blank="black"), 
                                   ci=False, #0.95
                                   picks=['Oz','O1','O2'], time_unit="ms", combine="mean")
fig[0].savefig("VEP Data/"+file_name+"-Compare_Stimuli")

combining channels using "mean"
combining channels using "mean"
